In [1]:
import cmd_options
import dataset

In [2]:
import sys
import shlex
import os.path

argv = """
--max_iter 1000000 \
--batch_size 128 \
--snapshot_step 5000 \
--test_step 250 \
--log_step 2 \
--dataset_name mpii
--train_csv_fn {0}/datasets/mpii/train_joints.csv \
--test_csv_fn {0}/datasets/mpii/test_joints.csv \
--val_csv_fn '' \
--img_path_prefix {0}/datasets/mpii/images \
--should_downscale_images \
--downscale_height 400 \
--n_joints 16 \
--seed 1701 \
--im_size 227 \
--min_dim 6 \
--shift 0.1 \
--bbox_extension_min 1.0 \
--bbox_extension_max 1.2 \
--coord_normalize \
--fname_index 0 \
--joint_index 1 \
--ignore_label -100500 \
--symmetric_joints "[[12, 13], [11, 14], [10, 15], [2, 3], [1, 4], [0, 5]]" \
--conv_lr 0.0005 \
--fc_lr 0.0005 \
--fix_conv_iter 0 \
--optimizer adagrad \
--o_dir {0}/out/mpii_alexnet_imagenet \
--gcn \
--fliplr \
--workers 4 \
--net_type Alexnet \
-s {0}/weights/bvlc_alexnet.tf \
--reset_iter_counter \
""".format('../deeppose_tf')

In [3]:
argv = shlex.split(argv)
print(argv)

['--max_iter', '1000000', '--batch_size', '128', '--snapshot_step', '5000', '--test_step', '250', '--log_step', '2', '--dataset_name', 'mpii', '--train_csv_fn', '../deeppose_tf/datasets/mpii/train_joints.csv', '--test_csv_fn', '../deeppose_tf/datasets/mpii/test_joints.csv', '--val_csv_fn', '', '--img_path_prefix', '../deeppose_tf/datasets/mpii/images', '--should_downscale_images', '--downscale_height', '400', '--n_joints', '16', '--seed', '1701', '--im_size', '227', '--min_dim', '6', '--shift', '0.1', '--bbox_extension_min', '1.0', '--bbox_extension_max', '1.2', '--coord_normalize', '--fname_index', '0', '--joint_index', '1', '--ignore_label', '-100500', '--symmetric_joints', '[[12, 13], [11, 14], [10, 15], [2, 3], [1, 4], [0, 5]]', '--conv_lr', '0.0005', '--fc_lr', '0.0005', '--fix_conv_iter', '0', '--optimizer', 'adagrad', '--o_dir', '../deeppose_tf/out/mpii_alexnet_imagenet', '--gcn', '--fliplr', '--workers', '4', '--net_type', 'Alexnet', '-s', '../deeppose_tf/weights/bvlc_alexnet.t

In [4]:
args = cmd_options.get_arguments(argv)
print(args)

Namespace(batch_size=128, bbox_extension_max=1.2, bbox_extension_min=1.0, conv_lr=0.0005, coord_normalize=True, dataset_name='mpii', downscale_height=400, fc_lr=0.0005, fix_conv_iter=0, fliplr=True, fname_index=0, gcn=True, ignore_label=-100500.0, im_size=227, img_path_prefix='../deeppose_tf/datasets/mpii/images', joint_index=1, log_step=2, max_iter=1000000, min_dim=6, n_joints=16, net_type='Alexnet', o_dir='../deeppose_tf/out/mpii_alexnet_imagenet', optimizer='adagrad', reset_iter_counter=True, reset_moving_averages=False, resume=False, rotate=False, rotate_range=10, seed=1701, shift=0.1, should_downscale_images=True, snapshot='../deeppose_tf/weights/bvlc_alexnet.tf', snapshot_step=5000, symmetric_joints='[[12, 13], [11, 14], [10, 15], [2, 3], [1, 4], [0, 5]]', test_csv_fn='../deeppose_tf/datasets/mpii/test_joints.csv', test_step=250, train_csv_fn='../deeppose_tf/datasets/mpii/train_joints.csv', val_csv_fn='', workers=4)


In [5]:
from __future__ import division
from chainer import iterators
import cmd_options
import dataset
import os
import time
#import regressionnet
import tensorflow as tf
import copy
from tqdm import tqdm
import numpy as np
import math
import pprint
import datetime

In [6]:
if not os.path.exists(args.o_dir):
    os.makedirs(args.o_dir)
suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
with open(os.path.join(args.o_dir, 'params.dump_{}.txt'.format(suffix)), 'w') as f:
    f.write('{}\n'.format(pprint.pformat(args)))

In [7]:
bbox_extension_range = (args.bbox_extension_min, args.bbox_extension_max)
if bbox_extension_range[0] is None or bbox_extension_range[1] is None:
    bbox_extension_range = None
    test_bbox_extension_range = None
else:
    test_bbox_extension_range = (bbox_extension_range[1], bbox_extension_range[1])

train_dataset = dataset.PoseDataset(
    args.train_csv_fn, args.img_path_prefix, args.im_size,
    fliplr=args.fliplr,
    rotate=args.rotate,
    rotate_range=args.rotate_range,
    shift=args.shift,
    bbox_extension_range=bbox_extension_range,
    min_dim=args.min_dim,
    coord_normalize=args.coord_normalize,
    gcn=args.gcn,
    fname_index=args.fname_index,
    joint_index=args.joint_index,
    symmetric_joints=args.symmetric_joints,
    ignore_label=args.ignore_label,
    should_downscale_images=args.should_downscale_images,
    downscale_height=args.downscale_height
)
test_dataset = dataset.PoseDataset(
    args.test_csv_fn, args.img_path_prefix, args.im_size,
    fliplr=False, rotate=False,
    shift=None,
    bbox_extension_range=test_bbox_extension_range,
    coord_normalize=args.coord_normalize,
    gcn=args.gcn,
    fname_index=args.fname_index,
    joint_index=args.joint_index,
    symmetric_joints=args.symmetric_joints,
    ignore_label=args.ignore_label,
    should_return_bbox=True,
    should_downscale_images=args.should_downscale_images,
    downscale_height=args.downscale_height
)

0it [00:00, ?it/s]

Reading dataset from ../deeppose_tf/datasets/mpii/train_joints.csv
Downscale images to the height 400px


2673it [00:52, 50.81it/s]/home/stonepeter/notebooks/deephead_tf/dataset.py:111: UserWarning: Skipping joint with incorrect joints coordinates. They are out of the image.
image: ../deeppose_tf/datasets/mpii/images/066231791.jpg, joint: [ 427.77777778  405.        ], im.shape: (400, 711)
  'image: {}, joint: {}, im.shape: {}'.format(img_path, joints[i_joint], image_shape[:2]))
3092it [00:59, 51.79it/s]/home/stonepeter/notebooks/deephead_tf/dataset.py:111: UserWarning: Skipping joint with incorrect joints coordinates. They are out of the image.
image: ../deeppose_tf/datasets/mpii/images/052008994.jpg, joint: [ 714.44444444  269.44444444], im.shape: (400, 711)
  'image: {}, joint: {}, im.shape: {}'.format(img_path, joints[i_joint], image_shape[:2]))
3454it [01:05, 52.53it/s]/home/stonepeter/notebooks/deephead_tf/dataset.py:111: UserWarning: Skipping joint with incorrect joints coordinates. They are out of the image.
image: ../deeppose_tf/datasets/mpii/images/072865628.jpg, joint: [ 278.888

Joints shape: (16, 2)
Reading dataset from ../deeppose_tf/datasets/mpii/test_joints.csv
Downscale images to the height 400px


818it [00:17, 47.86it/s]/home/stonepeter/notebooks/deephead_tf/dataset.py:111: UserWarning: Skipping joint with incorrect joints coordinates. They are out of the image.
image: ../deeppose_tf/datasets/mpii/images/091322969.jpg, joint: [ 508.14814815  405.92592593], im.shape: (400, 711)
  'image: {}, joint: {}, im.shape: {}'.format(img_path, joints[i_joint], image_shape[:2]))
1555it [00:31, 49.81it/s]/home/stonepeter/notebooks/deephead_tf/dataset.py:111: UserWarning: Skipping joint with incorrect joints coordinates. They are out of the image.
image: ../deeppose_tf/datasets/mpii/images/045623651.jpg, joint: [ 420.   452.5], im.shape: (400, 712)
  'image: {}, joint: {}, im.shape: {}'.format(img_path, joints[i_joint], image_shape[:2]))
1991it [00:39, 50.62it/s]

Joints shape: (16, 2)


In [9]:
import regressionnet
from regressionnet import evaluate_pcp, create_sumamry